# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data_df = pd.read_csv('../output_data/cities.csv')
weather_data_df.drop(labels='Unnamed: 0', axis = 1, inplace = True)
weather_data_df.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,punta arenas,CL,-53.15,-70.92,33.80,85,28,32.21,1597199569
1,cape town,ZA,-33.93,18.42,55.00,93,67,10.29,1597199482
2,ribeira grande,PT,38.52,-28.70,70.18,71,0,10.09,1597199569
3,longyearbyen,SJ,78.22,15.64,46.20,75,14,3.42,1597199569
4,dikson,RU,73.51,80.55,42.64,69,100,24.09,1597199570


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

In [25]:
city_list = weather_data_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_data_df['Humidity'].astype(float)

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(city_list, 
                                    weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
ideal_weather_df = weather_data_df

In [27]:
ideal_weather_df = ideal_weather_df[ideal_weather_df['Max Temp']>70]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Max Temp']<80]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed']<15]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness']<20]

ideal_weather_df.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,5.700000e+01
mean,26.565614,9.040351,75.292632,59.877193,2.789474,6.438947,1.597200e+09
std,25.290995,89.529314,2.830956,22.646405,5.041743,3.723304,9.814503e+01
min,-23.530000,-159.780000,70.180000,17.000000,0.000000,0.850000,1.597199e+09
25%,3.170000,-73.410000,72.250000,40.000000,0.000000,3.200000,1.597200e+09
50%,38.500000,9.660000,75.360000,60.000000,0.000000,5.500000,1.597200e+09
75%,42.730000,100.450000,77.580000,78.000000,3.000000,9.570000,1.597200e+09
max,60.370000,166.460000,79.470000,100.000000,18.000000,13.510000,1.597200e+09


In [29]:
ideal_weather_df.head(10)

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
2,ribeira grande,PT,38.52,-28.70,70.18,71,0,10.09,1597199569
10,atuona,PF,-9.80,-139.03,78.80,65,0,11.32,1597199573
64,avarua,CK,-21.21,-159.78,73.40,60,10,12.75,1597199524
83,guerrero negro,MX,27.98,-114.06,72.25,70,0,12.03,1597199602
87,camopi,GF,3.17,-52.32,71.56,96,2,2.46,1597199604
98,chara,RU,56.91,118.26,71.89,48,0,2.28,1597199610
104,iturama,BR,-19.73,-50.20,72.84,38,0,9.75,1597199612
123,mamakan,RU,57.82,114.00,78.35,44,17,1.52,1597199620
131,margate,GB,51.38,1.39,72.00,63,0,3.06,1597199617
137,destin,US,30.39,-86.50,75.20,88,1,5.82,1597199441


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = ideal_weather_df.copy()
url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
params={'radius':5000,
        'type': 'lodging',
        'key': g_key}

In [35]:
hotel_list = []

for index,row in hotel_df.iterrows():
    try:
        response = requests.get(url+str(row['Latitude'])+','+str(row['Longitude']),params=params).json()
        hotel_list.append(response['results'][0]['name'])
         
    except:
        hotel_list.append(None)
        
hotel_df['Hotel Name'] = hotel_list
hotel_df.head(10)

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
2,ribeira grande,PT,38.52,-28.70,70.18,71,0,10.09,1597199569,Quinta da Meia Eira
10,atuona,PF,-9.80,-139.03,78.80,65,0,11.32,1597199573,Villa Enata
64,avarua,CK,-21.21,-159.78,73.40,60,10,12.75,1597199524,Paradise Inn
83,guerrero negro,MX,27.98,-114.06,72.25,70,0,12.03,1597199602,"Casa Laguna, Bed & Breakfast"
87,camopi,GF,3.17,-52.32,71.56,96,2,2.46,1597199604,None
98,chara,RU,56.91,118.26,71.89,48,0,2.28,1597199610,None
104,iturama,BR,-19.73,-50.20,72.84,38,0,9.75,1597199612,República AmaZonas
123,mamakan,RU,57.82,114.00,78.35,44,17,1.52,1597199620,None
131,margate,GB,51.38,1.39,72.00,63,0,3.06,1597199617,Premier Inn Margate hotel
137,destin,US,30.39,-86.50,75.20,88,1,5.82,1597199441,Inn On Destin Harbor


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [44]:
# Add marker layer ontop of heat map

marker_locations= hotel_df[["Latitude","Longitude"]]
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(
    weather_data_df[["Latitude","Longitude"]], weights=weather_data_df["Humidity"], max_intensity = 150, point_radius = 10.0)
fig.add_layer(heatmap_layer)

markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))